# Airbyte API Testing for adding sources

----------------------------------------------------------------

This notebook's aim is to playaround and see how airbyte api works and how to make a python script to add sources.

In [1]:
# import statments
import requests
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
# airbyte api base url so that we don't keep typing it everytime

AIRBYTE_PUBLIC_API = "http://localhost:8000/api/public/v1/"
AIRBYTE_API = "http://localhost:8000/api/v1/"

In [24]:
# to access local environmental variables

load_dotenv(find_dotenv(), override=True)

API_KEY = os.getenv('BLINKMETRICS_OPENWEATHER_API_KEY')
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECERT = os.getenv('CLIENT_SECRET')
WORK_SPACE = os.getenv('WORK_SPACE_ID')
PASSWORD = os.getenv('DB_PASS')

Let us first test the connection with the API to see if it is accessible

In [4]:
# testing the connection to the API:

health_url = AIRBYTE_PUBLIC_API + '/health'

health_response = requests.get(health_url)
health_response.status_code

200

In [5]:
health_response.text # seeing the response to make sure the API connection is up

'Successful operation'

The API is accessible!

Now to get the token so we can access everything else in the API:

In [6]:
# getting the Token for authorization:

url_token = AIRBYTE_PUBLIC_API + "/applications/token"

payload = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECERT,
}

headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

token_response = requests.post(url_token, json=payload, headers=headers)

TOKEN = token_response.json()['access_token'] # storing the token in a constant

since we now got the token we can proceed with testing how to create a source :D

## Create a source:

**1. get the source definition:**

In [7]:
source_def_url = AIRBYTE_API + "/source_definitions/list" 

headers = {
    "accept": "application/json",
    "Content-Type" : "application/json",
    "authorization" : "Bearer " + TOKEN
} 

source_def_response = requests.post(source_def_url, headers=headers)
source_def_response.status_code

200

In [8]:
# getting the source def of OpenWeather so we can use it in the create source request 

for source_def in source_def_response.json()['sourceDefinitions']:
        if 'openweather' in source_def['name'].lower():
             SOURCE_DEF = source_def['sourceDefinitionId']
             
SOURCE_DEF

'561d7787-b45e-4f3b-af58-0163c3ba9d5a'

**2. create the source:**

In [9]:
url = AIRBYTE_API + "/sources/create"

payload = {
    "sourceDefinitionId": SOURCE_DEF,
    "workspaceId": WORK_SPACE,
    
    "connectionConfiguration": {
        "appid": API_KEY,
        "lat": '33.8959203',
        "lon": '35.47843'
    },

    "name": f"OpenWeather - Beirut - test"
}

headers = {
    "accept": "application/json",
    "Content-Type" : "application/json",
    "authorization" : "Bearer " + TOKEN
} 

response = requests.post(url, json=payload, headers=headers)
response.json()

{'sourceDefinitionId': '561d7787-b45e-4f3b-af58-0163c3ba9d5a',
 'sourceId': '5201d6c0-05db-4627-8134-b431a6a0b4d1',
 'workspaceId': 'b0841efb-2af8-410b-afe0-40a9837d3912',
 'connectionConfiguration': {'lat': '33.8959203',
  'lon': '35.47843',
  'appid': '**********'},
 'name': 'OpenWeather - Beirut - test',
 'sourceName': 'Openweather',
 'icon': 'https://connectors.airbyte.com/files/metadata/airbyte/source-openweather/latest/icon.svg',
 'isVersionOverrideApplied': False,
 'supportState': 'supported'}

seems like our test was successful, let us see if the source appears in the UI

<img src="..\assets\source_test_success.jpg" width="750"/>

the source did appear in the UI :D

now to try creating a destination 

## Create a Destination:

In [11]:
dest_def_url = AIRBYTE_API + "destination_definitions/list" 

headers = {
    "accept": "application/json",
    "Content-Type" : "application/json",
    "authorization" : "Bearer " + TOKEN
} 

dest_def_response = requests.post(dest_def_url, headers=headers)
dest_def_response.status_code

200

In [13]:
for destination_def in dest_def_response.json()['destinationDefinitions']:
    if 'MySQL'.lower() in destination_def['name'].lower():
        DEST_DEF = destination_def['destinationDefinitionId']

In [23]:

url = AIRBYTE_API + "/destinations/create"

payload = { 
    "destinationDefinitionId": DEST_DEF,
    "workspaceId": WORK_SPACE,

    "connectionConfiguration": {
        "destinationType": "mysql",
        "host": "host.docker.internal",
        "port": 3306,
        "database": "blinkmetrics_project_stage",
        "username": "root",
        "password": PASSWORD
    },

    "name": "MySQL - test"}

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": "Bearer " + TOKEN
}

response = requests.post(url, json=payload, headers=headers)

response.text

'{"destinationDefinitionId":"ca81ee7c-3163-4246-af40-094cc31e5e42","destinationId":"da8ae1aa-30e0-4487-87d8-f5c05b84421f","workspaceId":"b0841efb-2af8-410b-afe0-40a9837d3912","connectionConfiguration":{"host":"host.docker.internal","port":3306,"database":"blinkmetrics_project_stage","password":"**********","username":"root","destinationType":"mysql"},"name":"MySQL - test","destinationName":"MySQL","icon":"https://connectors.airbyte.com/files/metadata/airbyte/destination-mysql/latest/icon.svg","isVersionOverrideApplied":false,"supportState":"supported"}'

<img src="..\assets\destination_test_success.jpg" width="750"/>

This was a success too :D!

## Create a Connection:

In [39]:
# Define the endpoint URL for listing sources
url = AIRBYTE_API + "/sources/list"

payload = {
    "workspaceId": WORK_SPACE
}

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {TOKEN}" 
}

# Send the request to get the list of sources
response = requests.post(url, json=payload, headers=headers)

# Check the response status and print the result
if response.status_code == 200:
    sources = response.json()['sources']
    print(sources)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


[{'sourceDefinitionId': '55befeb8-1af2-4142-8f5e-13993f86741c', 'sourceId': '1bc629e8-b5f2-4f8a-b350-36abb3ab079c', 'workspaceId': 'b0841efb-2af8-410b-afe0-40a9837d3912', 'connectionConfiguration': {'api_key': '**********', '__injected_declarative_manifest': {}}, 'name': 'Open Weather Current', 'sourceName': 'Open Weather Current', 'isVersionOverrideApplied': False, 'supportState': 'supported', 'status': 'inactive'}, {'sourceDefinitionId': '561d7787-b45e-4f3b-af58-0163c3ba9d5a', 'sourceId': 'e25e436c-a7d9-4c24-b652-1dc67e6180a5', 'workspaceId': 'b0841efb-2af8-410b-afe0-40a9837d3912', 'connectionConfiguration': {'lat': '33.8959203', 'lon': '35.47843', 'appid': '**********'}, 'name': 'OpenWeather - Beirut - test', 'sourceName': 'Openweather', 'icon': 'https://connectors.airbyte.com/files/metadata/airbyte/source-openweather/latest/icon.svg', 'isVersionOverrideApplied': False, 'supportState': 'supported', 'status': 'inactive'}, {'sourceDefinitionId': '561d7787-b45e-4f3b-af58-0163c3ba9d5a',

In [43]:
source_list = []
for source in sources[2:]:
    source_list.append({source['sourceId'], source['name']})

print(source_list)

[{'2eec855f-4ed1-4f1a-a0c3-3c47c6e68b35', 'OpenWeather - Brasília'}, {'OpenWeather - Ottawa', '0a88c816-6861-40e3-8e04-3205c0734f99'}, {'2326289d-10a2-46ca-bf28-33919d0e29fa', 'OpenWeather - Paris'}, {'70b504a3-f9a6-4200-8c22-d7ad38fc4be1', 'OpenWeather - Berlin'}, {'18b0a126-67c0-4bfe-8bc5-c41ecb47c4bd', 'OpenWeather - Tehran'}, {'OpenWeather - Baghdad', '3fd74f73-d5f3-4079-ad9a-41bb4080399b'}, {'OpenWeather - Rome', 'aba0f704-2d2d-43a6-a9ac-4ca35f1ea024'}, {'OpenWeather - Amman', 'b358cc42-f217-45e1-b859-45a7b589d45d'}, {'OpenWeather - Beirut', 'e2be6309-4745-441c-9012-77fd3455d9c3'}, {'OpenWeather - Malé', '4400828f-89e3-4a80-9462-658766be0651'}, {'bfee574a-0921-4439-96a3-a4317eaf1f9f', 'OpenWeather - Moscow'}, {'feeb30c0-2586-4263-95c8-8bec731fe8e4', 'OpenWeather - Riyadh'}, {'9475e37a-c2be-48af-b7a6-334dcb616c65', 'OpenWeather - Pretoria'}, {'66566bfd-2599-434e-90c6-dd37710e5943', 'OpenWeather - Kyiv'}, {'OpenWeather - Abu Dhabi', '3c518e04-a8d3-4181-b78f-d6fd91077339'}, {'OpenWea

we have 17 sources, this is due to an issue with how airbyte is taking the lon it is supposed to be between -180 ad 180, however it is not accepting values about 99 and below -99

In [30]:
source_id = "e25e436c-a7d9-4c24-b652-1dc67e6180a5"  # Source ID you got when creating the source

url = AIRBYTE_API + "/sources/discover_schema"

payload = {
    "sourceId": source_id
}

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": "Bearer " + TOKEN
}

response = requests.post(url, json=payload, headers=headers)

schema_response = response.json()
jsonSchema = schema_response['catalog']['streams'][0]['stream']['jsonSchema']

In [42]:
jsonSchema

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'additionalProperties': True,
 'type': 'object',
 'properties': {'alerts': {'description': 'Weather alerts for the requested location',
   'type': 'array'},
  'current': {'description': 'Contains current weather data',
   'additionalProperties': True,
   'type': 'object',
   'properties': {'rain': {'description': 'Information about precipitation in the last hour',
     'additionalProperties': True,
     'type': 'object',
     'properties': {'1h': {'description': 'Rain volume for the last hour',
       'type': 'number'}}},
    'temp': {'description': 'Temperature', 'type': 'number'},
    'sunrise': {'description': 'Sunrise time', 'type': 'number'},
    'visibility': {'description': 'Visibility distance', 'type': 'number'},
    'uvi': {'description': 'UV index', 'type': 'number'},
    'clouds': {'description': 'Cloudiness percentage', 'type': 'number'},
    'pressure': {'description': 'Atmospheric pressure on the sea level',
     't

In [36]:
source_id = "e25e436c-a7d9-4c24-b652-1dc67e6180a5" 
destination_id = "da8ae1aa-30e0-4487-87d8-f5c05b84421f" 

url = AIRBYTE_API + "/connections/create"

payload = {
    "sourceId": source_id,
    "destinationId": destination_id,
    "name": "Beirut Weather to MySQL - test",
    "namespaceDefinition": "source",  
    "namespaceFormat": "${SOURCE_NAMESPACE}",  
    "syncCatalog": {
        "streams": [
            {
                "stream": {
                    "name": "onecall",  
                    "namespace": None,  
                    "jsonSchema": jsonSchema,
                    "supportedSyncModes": ["full_refresh", "incremental"]
                },
                "config": {
                    "syncMode": "incremental",
                    "cursorField": '_airbyte_extracted_at',  
                    "destinationSyncMode": "append",  
                    "selected": True
                }
            }
        ]
    },
    "scheduleType": "manual",  
    "status": "active"
}


headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": "Bearer " + TOKEN
}


response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'connectionId': '628c57f0-fc79-474c-8f46-cd5f1c16255c', 'name': 'Beirut Weather to MySQL - test', 'namespaceDefinition': 'source', 'namespaceFormat': '${SOURCE_NAMESPACE}', 'sourceId': 'e25e436c-a7d9-4c24-b652-1dc67e6180a5', 'destinationId': 'da8ae1aa-30e0-4487-87d8-f5c05b84421f', 'operationIds': [], 'syncCatalog': {'streams': [{'stream': {'name': 'onecall', 'jsonSchema': {'type': 'object', '$schema': 'http://json-schema.org/draft-07/schema#', 'properties': {'lat': {'type': 'number', 'description': 'Latitude of the requested location'}, 'lon': {'type': 'number', 'description': 'Longitude of the requested location'}, 'daily': {'type': 'array', 'description': 'Weather forecast for the next 7 days'}, 'alerts': {'type': 'array', 'description': 'Weather alerts for the requested location'}, 'hourly': {'type': 'array', 'description': 'Weather forecast for the next 48 hours'}, 'current': {'type': 'object', 'properties': {'dt': {'type': 'number', 'description': 'Time of the data forecasted'}, 

<img src="..\assets\connector_test_success.jpg" width="750"/>

looks like a success on this end! 

still have to see how the data is loaded in MySQL to make sure!

<img src="..\assets\load_test_success.jpg" width="750"/>

this was a success :D 

the EL part is now tested! onto the T part next :>